# Calibration
### Supply Chain Maze

In [12]:
import cv2
import numpy as np
# pip install Pillow
    # open Anaconda Prompt and paste above line (without '#') to install package
from PIL import Image

In [14]:
# function that gives a range on hues given a color
def get_limits(color):
    c = np.uint8([[color]])
    hsvC = cv2.cvtColor(c, cv2.COLOR_BGR2HSV)
    
    lowerLimit = hsvC[0][0][0] - 10, 100, 100
    upperLimit = hsvC[0][0][0] + 10, 255, 255
    # the +/-10 defines the range of hues that fall within the limits (the h in hsv)
    # the range on saturation and value is much bigger because we are only looking for hue
    
    lowerLimit = np.array(lowerLimit, dtype=np.uint8)
    upperLimit = np.array(upperLimit, dtype=np.uint8)

    return lowerLimit, upperLimit

How to use:
- place color marker on the node indicated on screen
- press [spacebar] to record that point
  - press [backspace] to undo
- once you have recorded all points, program ends and prints the points
- copy and paste the output into the "points" variable on the main Supply Chain Maze program

The points you find on this program will tell us the location of the nodes in the main SCM program, so that we can look around those points for markers and draw lines between nodes.
If you move the camera after using this program, the points will be inaccurate and you will need to recalibrate.
There are 9 points being recorded, start, end, and numbers 1-7.

In [24]:
color = [90, 144, 31]  # color in BGR colorspace
detection_size = 100              # size of detection required to mark color as an object
capture = cv2.VideoCapture(0)     # picks camera to use (usually 0 or 1)

first = False
height = 0
count = 0
nodes = list()
key = 0
while True:
    ret, frame = capture.read()
    frame = cv2.flip(frame, 1)
    
    if (first == False):
        shape = frame.shape
        height = shape[0]
        width = shape[1]
        cqm = width / 640  # scales detection with video quality (because it depends on number of pixels)
        first = True
    
    frame_blur = cv2.GaussianBlur(frame, (11, 11), 9) # blurring the image may help get the desired result, but it can be removed
    
    frame_hsv = cv2.cvtColor(frame_blur, cv2.COLOR_BGR2HSV) # convert to HSV
    lowerLimit, upperLimit = get_limits(color) # range of hues that we want the software to detect
    
    mask = cv2.inRange(frame_hsv, lowerLimit, upperLimit) # detects objects in color range
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    bboxes = list()
    for cnt in contours:
        if cv2.contourArea(cnt) > detection_size*cqm**2:  # only continues if size of the object is large enough (removes noise)
            x1, y1, w, h = cv2.boundingRect(cnt) # finds a bounding box for each object
            c = list([int(x1 + w/2), int(y1 + h/2)])  # centerpoint of bbox
            # check other bboxes to see if we want to combine them into 1 box
            newBox = list([c[0], c[1], x1, y1, w, h, cv2.contourArea(cnt)])
            for i in bboxes[:]:
                cxi, cyi, x1i, y1i, wi, hi, si = i  # bbox we check the newBox against
                if np.sqrt((c[0] - cxi)**2 + (c[1] - cyi)**2) < np.sqrt(w**2 + h**2)/3 + np.sqrt(wi**2 + hi**2)/3 + cqm*30: # if centerpoints are close enough (scales with box size)
                    bboxes.remove(i)
                    # reassign bbox boundaries so the new box contains both nearby boxes
                    newBox[2], newBox[3] = min(x1, x1i), min(y1, y1i)  # reassigns x1 and y1 values
                    newBox[4], newBox[5] = max(x1+w, x1i+wi) - newBox[2], max(y1+h, y1i+hi) - newBox[3]  # reassgins w and h values
                    newBox[0], newBox[1] = int(newBox[2] + newBox[4]/2), int(newBox[3] + newBox[5]/2)  # reassigns centerpoint values
            bboxes.append(newBox)

    # find the largest object
    maxVal = 0
    point = 0
    for i in bboxes:  
        cx, cy, x1, y1, w, h, s = i
        if s > maxVal:
            maxVal = s
            point = i

    # draw bounding box
    if point != 0:
        cx, cy, x1, y1, w, h, s = point
        cv2.rectangle(frame, (x1, y1), (x1 + w, y1 + h), (0, 0, 255), 2)
        # drawing dots at the centerpoint
        cv2.line(frame, (cx, cy), (cx, cy), (255, 0, 0), 15)
        cv2.line(frame, (cx, cy), (cx, cy), (0, 255, 255), 10)
        cv2.line(frame, (cx, cy), (cx, cy), (0, 0, 255), 5)

    # print instruction text
    text_color = (0, 0, 0)
    text_border_color = (255, 255, 255)
    if count == 0:
        cv2.putText(frame, 'Calibrating: Start', (int(24*cqm), int(height - 25*cqm)), cv2.FONT_HERSHEY_TRIPLEX, int(1*cqm), text_border_color, int(1*cqm))
        cv2.putText(frame, 'Calibrating: Start', (int(26*cqm), int(height - 25*cqm)), cv2.FONT_HERSHEY_TRIPLEX, int(1*cqm), text_border_color, int(1*cqm))
        cv2.putText(frame, 'Calibrating: Start', (int(25*cqm), int(height - 24*cqm)), cv2.FONT_HERSHEY_TRIPLEX, int(1*cqm), text_border_color, int(1*cqm))
        cv2.putText(frame, 'Calibrating: Start', (int(25*cqm), int(height - 26*cqm)), cv2.FONT_HERSHEY_TRIPLEX, int(1*cqm), text_border_color, int(1*cqm))
        cv2.putText(frame, 'Calibrating: Start', (int(25*cqm), int(height - 25*cqm)), cv2.FONT_HERSHEY_TRIPLEX, int(1*cqm), text_color, int(1*cqm))
    elif count == 8:
        cv2.putText(frame, 'Calibrating: End', (int(24*cqm), int(height - 25*cqm)), cv2.FONT_HERSHEY_TRIPLEX, int(1*cqm), text_border_color, int(1*cqm))
        cv2.putText(frame, 'Calibrating: End', (int(26*cqm), int(height - 25*cqm)), cv2.FONT_HERSHEY_TRIPLEX, int(1*cqm), text_border_color, int(1*cqm))
        cv2.putText(frame, 'Calibrating: End', (int(25*cqm), int(height - 24*cqm)), cv2.FONT_HERSHEY_TRIPLEX, int(1*cqm), text_border_color, int(1*cqm))
        cv2.putText(frame, 'Calibrating: End', (int(25*cqm), int(height - 26*cqm)), cv2.FONT_HERSHEY_TRIPLEX, int(1*cqm), text_border_color, int(1*cqm))
        cv2.putText(frame, 'Calibrating: End', (int(25*cqm), int(height - 25*cqm)), cv2.FONT_HERSHEY_TRIPLEX, int(1*cqm), text_color, int(1*cqm))
    else:
        cv2.putText(frame, f'Calibrating: Point {count}', (int(24*cqm), int(height - 25*cqm)), cv2.FONT_HERSHEY_TRIPLEX, int(1*cqm), text_border_color, int(1*cqm))
        cv2.putText(frame, f'Calibrating: Point {count}', (int(26*cqm), int(height - 25*cqm)), cv2.FONT_HERSHEY_TRIPLEX, int(1*cqm), text_border_color, int(1*cqm))
        cv2.putText(frame, f'Calibrating: Point {count}', (int(25*cqm), int(height - 24*cqm)), cv2.FONT_HERSHEY_TRIPLEX, int(1*cqm), text_border_color, int(1*cqm))
        cv2.putText(frame, f'Calibrating: Point {count}', (int(25*cqm), int(height - 26*cqm)), cv2.FONT_HERSHEY_TRIPLEX, int(1*cqm), text_border_color, int(1*cqm))
        cv2.putText(frame, f'Calibrating: Point {count}', (int(25*cqm), int(height - 25*cqm)), cv2.FONT_HERSHEY_TRIPLEX, int(1*cqm), text_color, int(1*cqm))
    
    
    # record locations of nodes
    if key == 32 and point != 0:   # when [spacebar] is pressed
        nodes.append([point[0], point[1]])
        point = 0
        maxVal = 0
        count += 1
        if count >= 9:
            break
        cv2.waitKey(250)    

    # removes last point if [backspace] is pressed
    if key == 8 and len(nodes) > 0:
        nodes.pop()
        count -= 1
        cv2.waitKey(250)
                 
    cv2.imshow('mask', mask)
    cv2.imshow('webcam', frame)

    key = cv2.waitKey(10) & 0xFF
    if key == 27:               # 27 is ASCII for [ESC]
        break                   # ends loop when [ESC] is pressed

capture.release()
cv2.destroyAllWindows()  # closes window, only reaches here when spacebar is pressed

# print points we found
print(nodes)

[[99, 235], [192, 142], [172, 322], [301, 114], [281, 221], [283, 342], [436, 106], [442, 326], [539, 210]]
